# Building steel coils

This tutorial includes everything you need to set up Decision Optimization engines, build constraint programming models.

**Table of contents:**

-  [Describe the business problem](#Describe-the-business-problem)
*  [How Decision Optimization can help](#How-Decision-Optimization-can-help)
*  [Use Decision Optimization](#Use-Decision-Optimization)
    *  [Step 1: Model the Data](#Step-1:-Model-the-data)
    *  [Step 2: Set up the prescriptive model](#Step-2:-Set-up-the-prescriptive-model)
        * [Define the decision variables](#Define-the-decision-variables)
        * [Express the business constraints](#Express-the-business-constraints)
        * [Express the objective](#Express-the-objective)
        * [Solve the model](#Solve-the-model)
    *  [Step 3: Investigate the solution and run an example analysis](#Step-3:-Investigate-the-solution-and-then-run-an-example-analysis)
*  [Summary](#Summary)
****

### Describe the business problem

* The problem is to build steel coils from slabs that are available in a work-in-process inventory of semi-finished products. There is no limitation in the number of slabs that can be requested, but only a finite number of slab sizes are available (sizes 11, 13, 16, 17, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 40, 43, 45). 
* The problem is to select a number of slabs to build the coil orders, and to satisfy the following constraints:
    * A coil order can be built from only one slab.
    * Each coil order requires a specific process to build it from a slab. This process is encoded by a color.
    * Several coil orders can be built from the same slab. But a slab can be used to produce at most two different "colors" of coils.
    * The sum of the sizes of each coil order built from a slab must not exceed the slab size.


* Finally, the production plan should minimize the unused capacity of the selected slabs.


* This problem is based on **prob038: Steel mill slab design problem** from <a href="http://www.csplib.org" target="_blank" rel="noopener noreferrer">CSPLib</a>. It is a simplification of an industrial problem described in J. R. Kalagnanam, M. W. Dawande, M. Trumbo, H. S. Lee. "Inventory Matching Problems in the Steel Industry," IBM Research Report RC 21171, 1998.

* Please refer to documentation for appropriate setup of solving configuration.

*****
## How Decision Optimization can help
* Prescriptive analytics (Decision Optimization) technology recommends actions based on desired outcomes, taking into account specific scenarios, resources, and knowledge of past and current events. This insight can help your organization make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

+ For example:
    + Automate complex decisions and trade-offs to better manage limited resources.
    + Take advantage of a future opportunity or mitigate a future risk.
    + Proactively update recommendations based on changing events.
    + Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.


## Use Decision Optimization

### Step 1: Model the data

In [1]:
from docplex.cp.model import *

Set model parameter

In [2]:
from collections import namedtuple

##############################################################################
# Model configuration
##############################################################################

# The number of coils to produce
TUPLE_ORDER = namedtuple("TUPLE_ORDER", ["index", "weight", "color"])
orders = [ TUPLE_ORDER(1, 22, 5),
           TUPLE_ORDER(2,  9, 3),
           TUPLE_ORDER(3,  9, 4),
           TUPLE_ORDER(4,  8, 5),
           TUPLE_ORDER(5,  8, 7),
           TUPLE_ORDER(6,  6, 3),
           TUPLE_ORDER(7,  5, 6),
           TUPLE_ORDER(8,  3, 0),
           TUPLE_ORDER(9,  3, 2),
           TUPLE_ORDER(10, 3, 3),
           TUPLE_ORDER(11, 2, 1),
           TUPLE_ORDER(12, 2, 5)
           ]

NB_SLABS = 12
MAX_COLOR_PER_SLAB = 2

# The total number of slabs available.  In theory this can be unlimited,
# but we impose a reasonable upper bound in order to produce a practical
# optimization model.

# The different slab weights available.
slab_weights = [ 0, 11, 13, 16, 17, 19, 20, 23, 24, 25,
                 26, 27, 28, 29, 30, 33, 34, 40, 43, 45 ]

In [3]:
nb_orders = len(orders)
slabs = range(NB_SLABS)
allcolors = set([ o.color for o in orders ])

# CPO needs lists for pack constraint
order_weights = [ o.weight for o in orders ]

# The heaviest slab
max_slab_weight = max(slab_weights)

# The amount of loss incurred for different amounts of slab use
# The loss will depend on how much less steel is used than the slab
# just large enough to produce the coils.
loss = [ min([sw-use for sw in slab_weights if sw >= use]) for use in range(max_slab_weight+1)]

### Step 2: Set up the prescriptive model

Create CPO model

In [4]:
mdl = CpoModel(name="trucks")

#### Define the decision variables

In [5]:
# Which slab is used to produce each coil
production_slab = integer_var_dict(orders, 0, NB_SLABS-1, "production_slab")

# How much of each slab is used
slab_use = integer_var_list(NB_SLABS, 0, max_slab_weight, "slab_use")

#### Express the business constraints

In [6]:
# The total loss is
total_loss = sum([element(slab_use[s], loss) for s in slabs])

# The orders are allocated to the slabs with capacity
mdl.add(pack(slab_use, [production_slab[o] for o in orders], order_weights))

# At most MAX_COLOR_PER_SLAB colors per slab
for s in slabs:
    su = 0
    for c in allcolors:
        lo = False
        for o in orders:
            if o.color==c:
                lo = (production_slab[o] == s) | lo
        su += lo
    mdl.add(su <= MAX_COLOR_PER_SLAB)

#### Express the objective

In [7]:
# Add minimization objective
mdl.add(minimize(total_loss))

#### Solve the model

In [8]:
print("\nSolving model....")
# Search strategy
mdl.set_search_phases([search_phase([production_slab[o] for o in orders])])

msol = mdl.solve(FailLimit=100000, TimeLimit=10)


Solving model....
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 24 variables, 13 constraints, 1 phase
 ! Presolve      : 24 extractables eliminated, 2 constraints generated
 ! FailLimit            = 100000
 ! TimeLimit            = 10
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 109.3 (before), 109.3 (after)
 !  . Memory usage      : 336.5 kB (before), 336.5 kB (after)
 ! Using parallel search with 32 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0         24                 -
 + New bound is 0
 *             0       18  0.20s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 0 (optimal - effective tol. is 0)
 ! Best bound

### Step 3: Investigate the solution and then run an example analysis

In [9]:
# Print solution
if msol:
    print("Solution: ")
    from_slabs = [set([o.index for o in orders if msol[production_slab[o]]== s])for s in slabs]
    slab_colors = [set([o.color for o in orders if o.index in from_slabs[s]])for s in slabs]
    for s in slabs:
        if len(from_slabs[s]) > 0:
            print("Slab = " + str(s))
            print("\tLoss = " + str(loss[msol[slab_use[s]]]))
            print("\tcolors = " + str(slab_colors[s]))
            print("\tOrders = " + str(from_slabs[s]) + "\n")
else:
    print("No solution found")

Solution: 
Slab = 0
	Loss = 0
	colors = {1, 3}
	Orders = {11, 2, 10, 6}

Slab = 1
	Loss = 0
	colors = {0, 5}
	Orders = {8, 1}

Slab = 2
	Loss = 0
	colors = {6, 7}
	Orders = {5, 7}

Slab = 3
	Loss = 0
	colors = {4, 5}
	Orders = {3, 12}

Slab = 4
	Loss = 0
	colors = {2, 5}
	Orders = {9, 4}



## Summary

You have learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to formulate and solve a Constraint Programming model.

#### References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=cpdaas" target="_blank" rel="noopener noreferrer">IBM Cloud Pak for Data as a Service documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx" target="_blank" rel="noopener noreferrer">IBM watsonx.ai documentation</a>


<hr>
Copyright © 2017-2025. This notebook and its source code are released under the terms of the MIT License.